<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/langugae_model_finetune_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

In [2]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 33.2 MB/s 
     |████████████████████████████████| 4.9 MB 67.1 MB/s 
     |████████████████████████████████| 212 kB 75.4 MB/s 
     |████████████████████████████████| 115 kB 72.6 MB/s 
     |████████████████████████████████| 163 kB 73.3 MB/s 
     |████████████████████████████████| 127 kB 74.9 MB/s 
     |████████████████████████████████| 6.6 MB 61.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import transformers

print(transformers.__version__)

4.22.2


In [4]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'val.txt'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-566f8875ae80b499/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(datasets["train"])

In [7]:
#"facebook/opt-125m"
model_checkpoint = "facebook/opt-125m"

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/651 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [10]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
# block_size = tokenizer.model_max_length
block_size = 128

In [12]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

In [15]:
from transformers import Trainer, TrainingArguments

In [16]:
#https://stackoverflow.com/questions/72680734/huggingface-trainer-only-doing-3-epochs-no-matter-the-trainingarguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-health-test-1",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    #output_dir='./results',
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [18]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 32663
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12249


Epoch,Training Loss,Validation Loss
1,2.810900,2.780823
2,2.632800,2.736286
3,2.514900,2.726319


Saving model checkpoint to opt-125m-health-test-1/checkpoint-500
Configuration saved in opt-125m-health-test-1/checkpoint-500/config.json
Model weights saved in opt-125m-health-test-1/checkpoint-500/pytorch_model.bin
Saving model checkpoint to opt-125m-health-test-1/checkpoint-1000
Configuration saved in opt-125m-health-test-1/checkpoint-1000/config.json
Model weights saved in opt-125m-health-test-1/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to opt-125m-health-test-1/checkpoint-1500
Configuration saved in opt-125m-health-test-1/checkpoint-1500/config.json
Model weights saved in opt-125m-health-test-1/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to opt-125m-health-test-1/checkpoint-2000
Configuration saved in opt-125m-health-test-1/checkpoint-2000/config.json
Model weights saved in opt-125m-health-test-1/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to opt-125m-health-test-1/checkpoint-2500
Configuration saved in opt-125m-health-test-1/checkpoint-

TrainOutput(global_step=12249, training_loss=2.688490499583193, metrics={'train_runtime': 3842.4179, 'train_samples_per_second': 25.502, 'train_steps_per_second': 3.188, 'total_flos': 6400936230912000.0, 'train_loss': 2.688490499583193, 'epoch': 3.0})

In [20]:
!zip -r opt_health_v1.zip opt-125m-health-test-1/

  adding: opt-125m-health-test-1/ (stored 0%)
  adding: opt-125m-health-test-1/checkpoint-8000/ (stored 0%)
  adding: opt-125m-health-test-1/checkpoint-8000/config.json (deflated 48%)
  adding: opt-125m-health-test-1/checkpoint-8000/trainer_state.json (deflated 73%)
  adding: opt-125m-health-test-1/checkpoint-8000/training_args.bin (deflated 48%)
  adding: opt-125m-health-test-1/checkpoint-8000/rng_state.pth (deflated 27%)
  adding: opt-125m-health-test-1/checkpoint-8000/optimizer.pt (deflated 11%)
  adding: opt-125m-health-test-1/checkpoint-8000/scheduler.pt (deflated 49%)
  adding: opt-125m-health-test-1/checkpoint-8000/pytorch_model.bin (deflated 8%)
  adding: opt-125m-health-test-1/checkpoint-5000/ (stored 0%)
  adding: opt-125m-health-test-1/checkpoint-5000/config.json (deflated 48%)
  adding: opt-125m-health-test-1/checkpoint-5000/trainer_state.json (deflated 69%)
  adding: opt-125m-health-test-1/checkpoint-5000/training_args.bin (deflated 48%)
  adding: opt-125m-health-test-1/ch

In [22]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("opt-125m-health-test-1/checkpoint-12000")

loading configuration file opt-125m-health-test-1/checkpoint-12000/config.json
Model config OPTConfig {
  "_name_or_path": "opt-125m-health-test-1/checkpoint-12000",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

loading weights file opt-125m-health-test-1/checkpoint-12000/pytorch_model.bin
All model checkpoint weights were used when initializing OPTForCausalLM.

All the weights of OPTForCausalLM were 

In [25]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', tokenizer=AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True), model="opt-125m-health-test-1/checkpoint-12000", do_sample=True, num_return_sequences=5)
generator("The woman worked as a")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/934b6a077313f3ee660a918a95313f5d0b136c5a/config.json
Model config OPTConfig {
  "_name_or_path": "facebook/opt-125m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/934b6a077313f3ee6

[{'generated_text': 'The woman worked as a dishwasher for 16 months before the Covid-19 pandemic. The'},
 {'generated_text': 'The woman worked as a healthcare professional in an outbreak hospital in Japan; she and her patients carried out'},
 {'generated_text': 'The woman worked as a technician at a hospital in Kuwait while studying abroad. She later began looking after'},
 {'generated_text': 'The woman worked as a registered nurse and was married at the time of her death. According to local'},
 {'generated_text': 'The woman worked as a nurse for a hospital where she had a stroke. She left her home in'}]

In [26]:
generator("The event has a ")

[{'generated_text': 'The event has a  significance to the 2019-nCoV and transmission dynamics of the virus.'},
 {'generated_text': 'The event has a  and is related mainly to the COVID-19 outbreak in Thailand, which'},
 {'generated_text': 'The event has a  (0.5) degree of deviation from SIR model that has a'},
 {'generated_text': 'The event has a  long delay, with two key cases having a delayed flight. This is a'},
 {'generated_text': 'The event has a  significance for the epidemic development. An excess of 10 cases from Wuhan'}]

In [27]:
generator("The wedding started with a ")

[{'generated_text': 'The wedding started with a  walk-around of the priest and bride. It lasted for about 1'},
 {'generated_text': 'The wedding started with a  service at Marawi City of the Philippines. This service provided a formal'},
 {'generated_text': 'The wedding started with a  wedding of five couples in a church in the area of Paris just 2'},
 {'generated_text': 'The wedding started with a  large ceremony, during which a mix of officiants, guests, and'},
 {'generated_text': 'The wedding started with a  guests breakfast with a reception at 1:30pm in the church of'}]

In [28]:
generator("The conference started with a ")

[{'generated_text': "The conference started with a  Q&A session highlighting some of the key points on the speakers'"},
 {'generated_text': 'The conference started with a  discussion about the research in human breast tissue and its prognosis. Prof'},
 {'generated_text': 'The conference started with a  event on February 7, 2020, and was subsequently cancelled. The focus'},
 {'generated_text': 'The conference started with a  keynote address by Prof. Hermann Reisch, founder of The Well'},
 {'generated_text': 'The conference started with a  talk by Sigmund Bachmann, a German psychiatrist dedicated to the'}]

In [30]:
#event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
generator(sick_txt)

Input length of input_ids is 25, but `max_length` is set to 21. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  be'},
 {'generated_text': 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  ensure'},
 {'generated_text': 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  give'},
 {'generated_text': 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  give'},
 {'generated_text': 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  be'}]

In [31]:
generator2 = pipeline('text-generation', max_new_tokens=250, tokenizer=AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True), model="opt-125m-health-test-1/checkpoint-12000",
                      do_sample=True, num_return_sequences=10)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/934b6a077313f3ee660a918a95313f5d0b136c5a/config.json
Model config OPTConfig {
  "_name_or_path": "facebook/opt-125m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/934b6a077313f3ee6

In [32]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
generator2(event_txt + sick_txt)

[{'generated_text': 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will  be another year of hopeful plans. Stay informed with live updates on the current COVID-19 outbreak and visit our coronavirus hub for more advice on prevention and treatment. Global organizations have already started making public health threats like deforestation, burning coal, and the poisoning of the health systems. In turn, they are also intensifying their efforts to fight climate change. Stay informed with live updates on the current COVID-19 outbreak and visit our coronavirus hub for more advice on prevention and treatme

In [ ]:
#https://huggingface.co/docs/transformers/v4.16.2/en/custom_datasets

In [37]:
!cp -r opt-125m-health-test-1/checkpoint-12000/ opt-125m-health-test-v1/

In [38]:
!zip -r opt_health_v1.zip opt-125m-health-test-v1/

  adding: opt-125m-health-test-v1/ (stored 0%)
  adding: opt-125m-health-test-v1/checkpoint-12000/ (stored 0%)
  adding: opt-125m-health-test-v1/checkpoint-12000/config.json (deflated 48%)
  adding: opt-125m-health-test-v1/checkpoint-12000/trainer_state.json (deflated 76%)
  adding: opt-125m-health-test-v1/checkpoint-12000/training_args.bin (deflated 48%)
  adding: opt-125m-health-test-v1/checkpoint-12000/rng_state.pth (deflated 27%)
  adding: opt-125m-health-test-v1/checkpoint-12000/optimizer.pt (deflated 11%)
  adding: opt-125m-health-test-v1/checkpoint-12000/scheduler.pt (deflated 49%)
  adding: opt-125m-health-test-v1/checkpoint-12000/pytorch_model.bin (deflated 8%)
